In [1]:
import pandas as pd
import numpy as np
import sqlalchemy
import os
import glob
import xlrd
import xlsxwriter
import openpyxl
import psycopg2
import urllib
import openpyxl
import time
import re
import warnings

import tkinter as tk
from tkinter import ttk, simpledialog, messagebox, Scrollbar, Frame
from datetime import datetime, timedelta

from configparser import ConfigParser
from tqdm import tqdm
from datetime import datetime
from openpyxl.styles import Font, Color, Alignment, Border, Side, NamedStyle, PatternFill
from openpyxl.worksheet.dimensions import ColumnDimension
from openpyxl import  load_workbook
from openpyxl.utils.cell import get_column_letter
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font, Color, Alignment, Border, Side, NamedStyle, PatternFill
from openpyxl.worksheet.dimensions import ColumnDimension
from urllib.parse import quote
from psycopg2.extensions import AsIs
from dateutil.relativedelta import relativedelta
from IPython.display import Markdown as md
from sqlalchemy import create_engine
from sqlalchemy.exc import OperationalError


pd.options.display.float_format = '{:.2f}'.format
pd.options.display.max_columns
pd.options.display.max_rows
warnings.filterwarnings("ignore")

#### FINnet 2.0 Engine

In [9]:
FINnet_2_engine=sqlalchemy.create_engine('mssql+pyodbc://SAL_User01:%s@172.16.22.25:1433/FINCORE_BRIDGE_DB?driver=ODBC+Driver+17+for+SQL+Server' % quote('Sal@123'))

In [10]:
gos_query = ''' with cte1 as( select distinct reportid,pan from [Fincore_Bridge_DB].[FINCORE_BRIDGE].[person_entity_branch]
                    where Report_Type='STR' and cast(SUBSTRING(fiu_batch_id,4,6)as bigint) between 202304 and 202312 and pan is not null
                    ) 
                 select c1.reportid, pan, gos.gosTag2,gos.gosTextNarration
                 from cte1 c1
                 left join [Fincore_Bridge_DB].[FINCORE_BRIDGE].[fingate_Gos] gos 
                 on c1.reportid = gos.reportId                              
'''
print(gos_query)

 with cte1 as( select distinct reportid,pan from [Fincore_Bridge_DB].[FINCORE_BRIDGE].[person_entity_branch]
                    where Report_Type='STR' and cast(SUBSTRING(fiu_batch_id,4,6)as bigint) between 202304 and 202312 and pan is not null
                    ) 
                 select c1.reportid, pan, gos.gosTag2,gos.gosTextNarration
                 from cte1 c1
                 left join [Fincore_Bridge_DB].[FINCORE_BRIDGE].[fingate_Gos] gos 
                 on c1.reportid = gos.reportId                              



In [11]:
gos_data = pd.read_sql(gos_query,FINnet_2_engine)

In [12]:
print('Total data:',gos_data.shape[0],'rows')

Total data: 186484 rows


In [13]:
gos_data.head(2)

,reportid,pan,gosTag2,gosTextNarration
0,548002918,ABBCS9161P,Transaction Monitoring,Rohan Mishra an Individual having address: Shi...
1,548002918,AAVCA9957E,Transaction Monitoring,Rohan Mishra an Individual having address: Shi...


In [14]:
pwd

'C:\\Users\\LTIM_10701023\\Desktop\\Ravi\\Risk_Individuals_Jan23_Aug24'

In [20]:
df_keywords = pd.read_csv(r"C:\Users\LTIM_10701023\Desktop\Ravi\Adhoc\TF_FATF_S\keywords.csv")
df_keywords = pd.DataFrame(df_keywords[['Category','Keyword']])

In [19]:
df_keywords

,Category,Keyword
0,Terror financing,Terror
1,Terror financing,Radioactive Substance
2,Terror financing,Nuclear Device
3,Terror financing,Explosives
4,Terror financing,Terror Financing
5,Terror financing,Terror Funding


In [21]:
# Add a space at the beginning and end of each individual word in df_keywords['Keyword']
df_keywords['Keyword'] = df_keywords['Keyword'].apply(lambda x: ' + '.join(f' {word.strip()} ' for word in x.split(' + ')))

In [22]:
df_keywords['Keyword'] = df_keywords['Keyword'].str.lower()

In [24]:
df_keywords

,Category,Keyword
0,Terror financing,terror
1,Terror financing,radioactive substance
2,Terror financing,nuclear device
3,Terror financing,explosives
4,Terror financing,terror financing
5,Terror financing,terror funding


In [25]:
# Define the keyword matching function
def is_keyword_in_gos(gos, keywords):
    if not isinstance(gos, str):  # Check if 'gos' is a string
        return False
    return all(keyword in gos.lower() for keyword in keywords.lower().split(' + '))

In [26]:
df_gos = gos_data

# Prepare to collect results
results = []

# Batch processing in chunks of 10000
batch_size = 10000

output_file = 'tagged_gos_output.csv'

# Initialize the CSV file with headers for the first batch
with open(output_file, 'w') as f:
    f.write('reportid, pan,  Categories\n')  # Write CSV header

for start in tqdm(range(0, len(df_gos), batch_size), desc="Processing Batches"):
    end = min(start + batch_size, len(df_gos))
    df_batch = df_gos.iloc[start:end]  # Get the current batch

    # Cross-join DataFrames (Cartesian Product) for the current batch
    df_cross = df_batch.assign(key=1).merge(df_keywords.assign(key=1), on='key').drop('key', axis=1)

    # Apply keyword checking
    df_cross['is_match'] = df_cross.apply(lambda row: is_keyword_in_gos(row['gosTextNarration'], row['Keyword']), axis=1)

    # Filter only rows where there was a match
    df_tagged = df_cross[df_cross['is_match']].copy()

    # Group by the 'gosTextNarration' column to collect all matching categories
    df_tagged = df_tagged.groupby(['reportid', 'pan'])['Category'].apply(lambda x: ', '.join(set(x))).reset_index()

    # Merge back to original 'Gos' DataFrame to retain all rows
    df_final = df_gos.merge(df_tagged, on=['reportid', 'pan'], how='left').fillna('No Tag')

    # Append results for the current batch
    results.append(df_final)  # Store the results for final concatenation
    
    # Append results to the CSV file
    #df_tagged.to_csv(output_file, mode='a', header=False, index=False)

# Concatenate all batch results into a single DataFrame
df_final = pd.concat(results, ignore_index=True)

# Display the first two rows of the final output
df_final.head(2)

Processing Batches: 100%|██████████████████████████████████████████████████████████████| 19/19 [00:21<00:00,  1.16s/it]


,reportid,pan,gosTag2,gosTextNarration,Category
0,548002918,ABBCS9161P,Transaction Monitoring,Rohan Mishra an Individual having address: Shi...,No Tag
1,548002918,AAVCA9957E,Transaction Monitoring,Rohan Mishra an Individual having address: Shi...,No Tag


In [28]:
df_final.shape

(3543196, 5)

In [32]:
df_tf = df_final[df_final['Category']!='No Tag']

In [33]:
df_tf.shape

(69, 5)

In [34]:
country = 'singapore'

In [70]:
df_country = pd.DataFrame({'Keyword': [country]})
df_country

,Keyword
0,singapore


### Tagging Singapore as Country

In [71]:
df_gos = df_tf

# Prepare to collect results
results = []

# Batch processing in chunks of 10000
batch_size = 10000

output_file = 'tagged_gos_output.csv'

# Initialize the CSV file with headers for the first batch
with open(output_file, 'w') as f:
    f.write('reportid, pan,  Categories\n')  # Write CSV header

for start in tqdm(range(0, len(df_gos), batch_size), desc="Processing Batches"):
    end = min(start + batch_size, len(df_gos))
    df_batch = df_gos.iloc[start:end]  # Get the current batch

    # Cross-join DataFrames (Cartesian Product) for the current batch
    df_cross = df_batch.assign(key=1).merge(df_country.assign(key=1), on='key').drop('key', axis=1)

    # Apply keyword checking
    df_cross['is_match'] = df_cross.apply(lambda row: is_keyword_in_gos(row['gosTextNarration'], row['Keyword']), axis=1)

    # Filter only rows where there was a match
    df_tagged = df_cross[df_cross['is_match']].copy()

    # Group by the 'gosTextNarration' column to collect all matching categories
    df_tagged = df_tagged.groupby(['reportid', 'pan'])['Category'].apply(lambda x: ', '.join(set(x))).reset_index()

    # Merge back to original 'Gos' DataFrame to retain all rows
    df_final = df_gos.merge(df_tagged, on=['reportid', 'pan'], how='left').fillna('No Tag')

    # Append results for the current batch
    results.append(df_final)  # Store the results for final concatenation
    
    # Append results to the CSV file
    #df_tagged.to_csv(output_file, mode='a', header=False, index=False)

# Concatenate all batch results into a single DataFrame
df_final = pd.concat(results, ignore_index=True)

# Display the first two rows of the final output
df_final.head(2)

Processing Batches: 100%|████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 83.56it/s]


,reportid,pan,gosTag2,gosTextNarration,Category_x,Category_y
0,517889401,ABDPI0878C,Watch List,Gholam Md Izhar an Individual having address :...,Terror financing,No Tag
1,517889401,BRDPB2818N,Watch List,Gholam Md Izhar an Individual having address :...,Terror financing,No Tag


In [72]:
df_final.rename(columns={'Category_x':'Tagging','Category_y':'Country'},inplace = True)
df_final.shape[0]

69

### Searching Singapore in Whole STR for the given Period

In [51]:
df_keywords

,Category,Keyword
0,Terror financing,terror
1,Terror financing,radioactive substance
2,Terror financing,nuclear device
3,Terror financing,explosives
4,Terror financing,terror financing
5,Terror financing,terror funding


In [46]:
df_country

,Keyword
0,singapore


In [62]:
country = 'singapore'
df_country = pd.DataFrame({'Keyword': [country],'Category':'SIngapore'})
df_country

,Keyword,Category
0,singapore,SIngapore


In [63]:
df_gos = gos_data

# Prepare to collect results
results = []

# Batch processing in chunks of 10000
batch_size = 10000

output_file = 'tagged_gos_output.csv'

# Initialize the CSV file with headers for the first batch
with open(output_file, 'w') as f:
    f.write('reportid, pan,  Categories\n')  # Write CSV header

for start in tqdm(range(0, len(df_gos), batch_size), desc="Processing Batches"):
    end = min(start + batch_size, len(df_gos))
    df_batch = df_gos.iloc[start:end]  # Get the current batch

    # Cross-join DataFrames (Cartesian Product) for the current batch
    df_cross = df_batch.assign(key=1).merge(df_country.assign(key=1), on='key').drop('key', axis=1)

    # Apply keyword checking
    df_cross['is_match'] = df_cross.apply(lambda row: is_keyword_in_gos(row['gosTextNarration'], row['Keyword']), axis=1)

    # Filter only rows where there was a match
    df_tagged = df_cross[df_cross['is_match']].copy()

    # Group by the 'gosTextNarration' column to collect all matching categories
    df_tagged = df_tagged.groupby(['reportid', 'pan'])['Category'].apply(lambda x: ', '.join(set(x))).reset_index()

    # Merge back to original 'Gos' DataFrame to retain all rows
    df_final = df_gos.merge(df_tagged, on=['reportid', 'pan'], how='left').fillna('No Tag')

    # Append results for the current batch
    results.append(df_final)  # Store the results for final concatenation
    
    # Append results to the CSV file
    #df_tagged.to_csv(output_file, mode='a', header=False, index=False)

# Concatenate all batch results into a single DataFrame
df_final = pd.concat(results, ignore_index=True)

# Display the first two rows of the final output
df_final.head(2)

Processing Batches: 100%|██████████████████████████████████████████████████████████████| 19/19 [00:05<00:00,  3.24it/s]


,reportid,pan,gosTag2,gosTextNarration,Category
0,548002918,ABBCS9161P,Transaction Monitoring,Rohan Mishra an Individual having address: Shi...,No Tag
1,548002918,AAVCA9957E,Transaction Monitoring,Rohan Mishra an Individual having address: Shi...,No Tag


In [64]:
df_final.shape[0]

3543196

In [69]:
df_singapore = df_final[df_final['Category'] != 'No Tag']
print(df_singapore.shape[0])
display(df_singapore.head(2))

1231


,reportid,pan,gosTag2,gosTextNarration,Category
473,529446502,ASKPK6687P,Law Enforcement Agency Query,CUSTOMER KYC \tFull name and address of the Ac...,SIngapore
474,529446502,ABIFM6996L,Law Enforcement Agency Query,CUSTOMER KYC \tFull name and address of the Ac...,SIngapore


In [73]:
df_singapore.rename(columns={'Category':'Country_Tagged'}, inplace= True)

In [75]:
df_singapore.head(2)

,reportid,pan,gosTag2,gosTextNarration,Country_Tagged
473,529446502,ASKPK6687P,Law Enforcement Agency Query,CUSTOMER KYC \tFull name and address of the Ac...,SIngapore
474,529446502,ABIFM6996L,Law Enforcement Agency Query,CUSTOMER KYC \tFull name and address of the Ac...,SIngapore


In [76]:
df_gos = df_singapore

# Prepare to collect results
results = []

# Batch processing in chunks of 10000
batch_size = 10000

output_file = 'tagged_gos_output.csv'

# Initialize the CSV file with headers for the first batch
with open(output_file, 'w') as f:
    f.write('reportid, pan,  Categories\n')  # Write CSV header

for start in tqdm(range(0, len(df_gos), batch_size), desc="Processing Batches"):
    end = min(start + batch_size, len(df_gos))
    df_batch = df_gos.iloc[start:end]  # Get the current batch

    # Cross-join DataFrames (Cartesian Product) for the current batch
    df_cross = df_batch.assign(key=1).merge(df_keywords.assign(key=1), on='key').drop('key', axis=1)

    # Apply keyword checking
    df_cross['is_match'] = df_cross.apply(lambda row: is_keyword_in_gos(row['gosTextNarration'], row['Keyword']), axis=1)

    # Filter only rows where there was a match
    df_tagged = df_cross[df_cross['is_match']].copy()

    # Group by the 'gosTextNarration' column to collect all matching categories
    df_tagged = df_tagged.groupby(['reportid', 'pan'])['Category'].apply(lambda x: ', '.join(set(x))).reset_index()

    # Merge back to original 'Gos' DataFrame to retain all rows
    df_final = df_gos.merge(df_tagged, on=['reportid', 'pan'], how='left').fillna('No Tag')

    # Append results for the current batch
    results.append(df_final)  # Store the results for final concatenation
    
    # Append results to the CSV file
    #df_tagged.to_csv(output_file, mode='a', header=False, index=False)

# Concatenate all batch results into a single DataFrame
df_final = pd.concat(results, ignore_index=True)

# Display the first two rows of the final output
df_final.head(2)

Processing Batches: 100%|████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  5.79it/s]


,reportid,pan,gosTag2,gosTextNarration,Country_Tagged,Category
0,529446502,ASKPK6687P,Law Enforcement Agency Query,CUSTOMER KYC \tFull name and address of the Ac...,SIngapore,No Tag
1,529446502,ABIFM6996L,Law Enforcement Agency Query,CUSTOMER KYC \tFull name and address of the Ac...,SIngapore,No Tag


In [77]:
df_final.shape

(1231, 6)

In [78]:
df_final['Category'].unique()

array(['No Tag'], dtype=object)